# ML HW4 Sample Code
TODO:
 - Design your LSTM model
 - Use unlabelled data (train_nolabel.csv) for Word2Vec training
    - Combine labeled + unlabeled data to train better embeddings
 - Train with labelled data (train_label.csv)
    - Optional: Data augmentation
    - Optional: Custom loss function

## Download data

In [26]:
# !pip install -U gdown -q
# !gdown --folder https://drive.google.com/drive/folders/1786AXJRAtqFvWMBeh-bLm4MtU21IQpBg

## Import packages

In [27]:
import torch
import os
import csv
import random
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

# !pip install gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

## Set the Configurations

In [43]:
# Training Config
DEVICE_NUM = 2
BATCH_SIZE = 128
EPOCH_NUM = 20
MAX_POSITIONS_LEN = 500
SEED = 2025
MODEL_DIR = 'model.pth'
lr = 0.001

# Set Seed
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

# torch.cuda.set_device(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# RNN Config
w2v_config = {'path': 'w2v_model', 'dim': 512}
lstm_config = {'hidden_dim': 512, 'num_layers': 2, 'bidirectional': True, 'fix_embedding': True}
header_config = {'dropout': 0.3, 'hidden_dim': 1024}
assert header_config['hidden_dim'] == lstm_config['hidden_dim'] or header_config['hidden_dim'] == lstm_config['hidden_dim'] * 2

train_dataset_dropout = 0.2

Using device: cuda


## Utils for Datasets and Dataloaders

In [29]:
def parsing_text(text):
    # Explicitly handle None values
    if text is None:
        return ""
    # Handle NaN values (which often appear as floats in pandas)
    if pd.isna(text):
        return ""  # Convert NaN to an empty string
    # Optional: Add data preprocessing (e.g., lowercasing, removing special characters)
    text = text.lower()
    special_chars = ['!', '?', '(', ')', '[', ']', '{', '}', '"', "'", '_', '/', '\\', '@', '#', '$', '%', '^', '&', '*', '+', '=', '<', '>', '~', '`']
    for char in special_chars:
        text = text.replace(char, '')
    return str(text)

def load_train_label(path='train_label.csv'):
    tra_lb_pd = pd.read_csv(path)
    idx = tra_lb_pd['id'].tolist()
    text = [parsing_text(s).split() for s in tra_lb_pd['text'].tolist()]
    label = tra_lb_pd['label'].tolist()
    return idx, text, label

def load_train_nolabel(path='train_nolabel.csv'):
    tra_nlb_pd = pd.read_csv(path)
    text = [parsing_text(s).split() for s in tra_nlb_pd['text'].tolist()]
    return text

def load_test(path='test.csv'):
    test_pd = pd.read_csv(path)
    idx = test_pd['id'].tolist()
    text = [parsing_text(s).split() for s in test_pd['text'].tolist()]
    return idx, text

## Datasets and Dataloaders

In [44]:
class Preprocessor:
    def __init__(self, sentences, w2v_config):
        self.sentences = sentences
        self.idx2word = []
        self.word2idx = {}
        self.embedding_matrix = []
        self.build_word2vec(sentences, **w2v_config)

    def build_word2vec(self, x, path, dim):
        if os.path.isfile(path):
            print("loading word2vec model ...")
            w2v_model = Word2Vec.load(path)
        else:
            print("training word2vec model ...")
            w2v_model = Word2Vec(x, vector_size=dim, window=5, min_count=2, workers=12, epochs=10, sg=1)
            print("saving word2vec model ...")
            w2v_model.save(path)

        self.embedding_dim = w2v_model.vector_size
        for i, word in enumerate(w2v_model.wv.key_to_index):
            #e.g. self.word2index['he'] = 1
            #e.g. self.index2word[1] = 'he'
            #e.g. self.vectors[1] = 'he' vector

            self.word2idx[word] = len(self.word2idx)
            self.idx2word.append(word)
            self.embedding_matrix.append(w2v_model.wv[word])

        self.embedding_matrix = torch.tensor(self.embedding_matrix)
        self.add_embedding('<PAD>')
        self.add_embedding('<UNK>')
        print("total words: {}".format(len(self.embedding_matrix)))

    def add_embedding(self, word):
        vector = torch.empty(1, self.embedding_dim)
        torch.nn.init.uniform_(vector)
        self.word2idx[word] = len(self.word2idx)
        self.idx2word.append(word)
        self.embedding_matrix = torch.cat([self.embedding_matrix, vector], 0)

    def sentence2idx(self, sentence):
        sentence_idx = []
        for word in sentence:
            if word in self.word2idx.keys():
                sentence_idx.append(self.word2idx[word])
            else:
                sentence_idx.append(self.word2idx["<UNK>"])
        return torch.LongTensor(sentence_idx)

class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, id_list, sentences, labels, preprocessor, dropout=0.0):
        self.id_list = id_list
        self.sentences = sentences
        self.labels = labels
        self.preprocessor = preprocessor
        self.dropout = dropout
        
    def __getitem__(self, idx):
        if self.labels is None: return self.id_list[idx], self.preprocessor.sentence2idx(self.sentences[idx])
        if self.dropout > 0.0:
            sentence_idx = self.preprocessor.sentence2idx(self.sentences[idx])
            keep_mask = (torch.rand(len(sentence_idx)) > self.dropout).long()
            sentence_idx = sentence_idx * keep_mask + self.preprocessor.word2idx['<PAD>'] * (1 - keep_mask)
            return self.id_list[idx], sentence_idx, self.labels[idx]

        return self.id_list[idx], self.preprocessor.sentence2idx(self.sentences[idx]), self.labels[idx]

    def __len__(self):
        return len(self.sentences)

    def collate_fn(self, data):
        id_list = torch.LongTensor([d[0] for d in data])
        lengths = torch.LongTensor([len(d[1]) for d in data])
        texts = pad_sequence(
            [d[1] for d in data], batch_first=True).contiguous()

        if self.labels is None:
            return id_list, lengths, texts

        labels = torch.FloatTensor([d[2] for d in data])
        return id_list, lengths, texts, labels

## RNN Backbone

In [31]:
class LSTM_Backbone(torch.nn.Module):
    def __init__(self, embedding, hidden_dim, num_layers, bidirectional, fix_embedding=True):
        super(LSTM_Backbone, self).__init__()
        self.embedding = torch.nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = torch.nn.Parameter(embedding)
        self.embedding.weight.requires_grad = False if fix_embedding else True

        self.lstm = torch.nn.LSTM(embedding.size(1), hidden_dim, num_layers=num_layers, \
                                  bidirectional=bidirectional, batch_first=True)

    def forward(self, inputs):
        inputs = self.embedding(inputs)
        x, _ = self.lstm(inputs)
        return x

class Header(torch.nn.Module):
    def __init__(self, dropout, hidden_dim):
        super(Header, self).__init__()
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(dropout),
            torch.nn.Linear(hidden_dim, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(128, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )

    @torch.no_grad()
    def _get_length_masks(self, lengths):
        # lengths: (batch_size, ) in cuda
        ascending = torch.arange(MAX_POSITIONS_LEN)[:lengths.max().item()].unsqueeze(
            0).expand(len(lengths), -1).to(lengths.device)
        length_masks = (ascending < lengths.unsqueeze(-1)).unsqueeze(-1)
        return length_masks

    def forward(self, inputs, lengths):
        # the input shape should be (N, L, D∗H)
        pad_mask = self._get_length_masks(lengths)
        inputs = inputs * pad_mask
        inputs = inputs.sum(dim=1)
        out = self.classifier(inputs).squeeze()
        return out

## Training & Validation

In [32]:
def train(train_loader, backbone, header, optimizer, criterion, device, epoch):

    total_loss = []
    total_acc = []

    for i, (idx_list, lengths, texts, labels) in enumerate(train_loader):
        lengths, inputs, labels = lengths.to(device), texts.to(device), labels.to(device)

        optimizer.zero_grad()
        if not backbone is None:
            inputs = backbone(inputs)
        soft_predicted = header(inputs, lengths)
        loss = criterion(soft_predicted, labels)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(backbone.parameters(), max_norm=1.0) if not backbone is None else None
        # torch.nn.utils.clip_grad_norm_(header.parameters(), max_norm=1.0)
        optimizer.step()

        with torch.no_grad():
            hard_predicted = (soft_predicted >= 0.5).int()
            correct = sum(hard_predicted == labels).item()
            batch_size = len(labels)
            total_loss.append(loss.item())
            total_acc.append(correct * 100 / batch_size)
            print('[ Epoch {}: {}/{} ] loss:{:.3f} acc:{:.3f} '.format(epoch+1, i+1, len(train_loader), np.mean(total_loss), np.mean(total_acc)), end='\r')
    
    return np.mean(total_loss), np.mean(total_acc)


def valid(valid_loader, backbone, header, criterion, device, epoch):
    backbone.eval()
    header.eval()
    with torch.no_grad():
        total_loss = []
        total_acc = []

        for i, (idx_list, lengths, texts, labels) in enumerate(valid_loader):
            lengths, inputs, labels = lengths.to(device), texts.to(device), labels.to(device)

            if not backbone is None:
                inputs = backbone(inputs)
            soft_predicted = header(inputs, lengths)
            loss = criterion(soft_predicted, labels)
            total_loss.append(loss.item())

            hard_predicted = (soft_predicted >= 0.5).int()
            correct = sum(hard_predicted == labels).item()
            acc = correct * 100 / len(labels)
            total_acc.append(acc)

            print('[Validation in epoch {:}] loss:{:.3f} acc:{:.3f}'.format(epoch+1, np.mean(total_loss), np.mean(total_acc)), end='\r')
    backbone.train()
    header.train()
    return np.mean(total_loss), np.mean(total_acc)


def run_training(train_loader, valid_loader, backbone, header, epoch_num, lr, device, model_dir):
    best_acc = 0.0
    patience = 5
    counter = 0

    def check_point(backbone, header, loss, acc, model_dir):
        nonlocal best_acc
        if acc > best_acc:
            # Save state_dict instead of full model objects
            torch.save({
                'backbone_state_dict': backbone.state_dict(),
                'header_state_dict': header.state_dict()
            }, model_dir)
            print(f'New best model saved with accuracy: {acc:.3f}')

    def is_stop(loss, acc):
        # TODO: Implement early stopping
        nonlocal best_acc, counter, patience
        if acc > best_acc:
            counter = 0
            best_acc = acc
            return False
        else:
            counter += 1
            if counter >= patience:
                return True
        return False

    if backbone is None:
        trainable_paras = header.parameters()
    else:
        trainable_paras = list(backbone.parameters()) + list(header.parameters())

    optimizer = torch.optim.AdamW(trainable_paras, lr=lr, weight_decay=1e-3)
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

    backbone.train()
    header.train()
    backbone = backbone.to(device)
    header = header.to(device)
    criterion = torch.nn.BCELoss()

    for epoch in range(epoch_num):
        train_loss, train_acc = train(train_loader, backbone, header, optimizer, criterion, device, epoch)
        loss, acc = valid(valid_loader, backbone, header, criterion, device, epoch)
        print('[Epoch {:}] Train loss:{:.3f} acc:{:.3f} | Val loss:{:.3f} acc:{:.3f} | LR:{:.6f}'.format(
            epoch+1, train_loss, train_acc, loss, acc, optimizer.param_groups[0]['lr']))
        
        check_point(backbone, header, loss, acc, model_dir)
        scheduler.step(acc)
        
        if is_stop(loss, acc):
            print(f'Early stopping triggered after {epoch+1} epochs due to no improvement in accuracy for {patience} epochs.')
            break
    
    # Load best model for testing
    checkpoint = torch.load(model_dir, weights_only=True)
    backbone.load_state_dict(checkpoint['backbone_state_dict'])
    header.load_state_dict(checkpoint['header_state_dict'])
    print(f'Loaded best model with accuracy: {best_acc:.3f}')

## Testing

In [33]:
def run_testing(test_loader, backbone, header, device, output_path):
    with open(output_path, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['id', 'label'])

        with torch.no_grad():
            for i, (idx_list, lengths, texts) in enumerate(test_loader):
                lengths, inputs = lengths.to(device), texts.to(device)
                if not backbone is None:
                    inputs = backbone(inputs)
                soft_predicted = header(inputs, lengths)
                hard_predicted = (soft_predicted >= 0.5).int()
                for i, p in zip(idx_list, hard_predicted):
                    writer.writerow([str(i.item()), str(p.item())])

## Main

In [45]:
# Split Dataset
train_idx, train_label_text, label = load_train_label('dataset/train_label.csv')
train_nolabel_text = load_train_nolabel('dataset/train_nolabel.csv')

# For sanity check
print(train_label_text[:10])
print(train_nolabel_text[:10])

w2v_sentences = train_label_text + train_nolabel_text

# Use labeled data for Word2Vec embeddings (# TODO: Perform unsupervised Learning for w2v)
preprocessor = Preprocessor(w2v_sentences, w2v_config)

train_idx, valid_idx, train_label_text, valid_label_text, train_label, valid_label = train_test_split(train_idx, train_label_text, label, test_size=0.12)
train_dataset, valid_dataset = TwitterDataset(train_idx, train_label_text, train_label, preprocessor, train_dataset_dropout), TwitterDataset(valid_idx, valid_label_text, valid_label, preprocessor)

test_idx, test_text = load_test('dataset/test.csv')
test_dataset = TwitterDataset(test_idx, test_text, None, preprocessor)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True,
                                            collate_fn = train_dataset.collate_fn,
                                            num_workers = 8)
valid_loader = torch.utils.data.DataLoader(dataset = valid_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = False,
                                            collate_fn = valid_dataset.collate_fn,
                                            num_workers = 8)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = False,
                                            collate_fn = test_dataset.collate_fn,
                                            num_workers = 8)

# Instantiate The Model
backbone = LSTM_Backbone(preprocessor.embedding_matrix, **lstm_config)
header = Header(**header_config)

# Run Training
run_training(train_loader, valid_loader, backbone, header, EPOCH_NUM, lr, device, MODEL_DIR)

# Run Testing
run_testing(test_loader, backbone, header, device, 'pred.csv')

[['it', 'was', 'odd', 'not', 'walking', 'with', 'this', 'morning', '.'], ['doesnt', 'understand', 'how', 'a', 'man', 'can', 'pursue', 'a', 'woman', 'so', 'hard', ',', 'but', 'turns', 'around', 'and', 'cheats', '...', 'sigh'], ['i', 'm', 'tired', 'didn', 't', 'do', 'anything', 'all', 'day', 'except', 'went', 'to', 'the', 'craft', 'store', 'to', 'get', 'some', 'hemp', 'string'], ['going', 'to', 'cheddars'], ['haha', 'traci', 's', 'waaaaaasted', '.', 'i', 'miss', 'college'], ['fuck', 'after', 'yesterday', ',', 'whats', 'the', 'point', 'in', 'chasing', 'after', 'her', 'anymore', '....', 'for', 'once', 'in', 'my', 'life', 'i', 'guess', 'its', 'just', 'not', 'ment', 'to', 'be'], ['hey', 'mama', 'how', 'was', 'the', 'vacay'], ['well', 'donnee', 'woo', 'top', '10'], ['how', 'sweet', '.', 'but', 'you', 'dont', 'owe', 'me', 'anything', '.', 'thats', 'wat', 'friends', 'are', 'for', 'right'], ['you', 'may', 'be', 'right', 'i', 'll', 'just', 'have', 'to', 'have', 'my', 'team', 'look', 'to', 'other'

# Pseudo Labeling

In [46]:
train_idx, train_label_text, label = load_train_label('dataset/train_label.csv')
train_nolabel_text = load_train_nolabel('dataset/train_nolabel.csv')

w2v_sentences = train_label_text + train_nolabel_text
preprocessor = Preprocessor(w2v_sentences, w2v_config)

nolabel_dataset = TwitterDataset(list(range(len(train_nolabel_text))), train_nolabel_text, None, preprocessor)
nolabel_loader = torch.utils.data.DataLoader(nolabel_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=nolabel_dataset.collate_fn, num_workers=8)

backbone = LSTM_Backbone(preprocessor.embedding_matrix, **lstm_config)
header = Header(**header_config)

checkpoint = torch.load(MODEL_DIR, weights_only=True)
backbone.load_state_dict(checkpoint['backbone_state_dict'])
header.load_state_dict(checkpoint['header_state_dict'])

backbone.eval()
header.eval()

backbone = backbone.to(device)
header = header.to(device)

soft_threshold = 0.9
count = 0
pseudo_idx = 200000
pseudo_labels = []


with torch.no_grad():
    for i, (idx_list, lengths, texts) in enumerate(nolabel_loader):
        lengths, inputs = lengths.to(device), texts.to(device)
        if not backbone is None:
            inputs = backbone(inputs)
        soft_predicted = header(inputs, lengths)
        probs = soft_predicted.cpu()
        mask_pos = (probs > 0.9) & (random.random() < 0.2)
        mask_neg = (probs < 0.1) & (random.random() < 0.2)
        batched_pos_idx = idx_list[mask_pos].tolist()
        batched_neg_idx = idx_list[mask_neg].tolist()
        for idx in batched_pos_idx:
            pseudo_labels.append([pseudo_idx, 1])
        for idx in batched_neg_idx:
            pseudo_labels.append([pseudo_idx, 0])
            
with open('pseudo_labels.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'label', 'text'])
    rows_to_write = []
    for idx, label in pseudo_labels:
        original_text = ' '.join(train_nolabel_text[idx])
        rows_to_write.append([str(pseudo_idx + count), str(label), original_text])
        count += 1

        if len(rows_to_write) >= 1000:
            writer.writerows(rows_to_write)
            rows_to_write = []

    if rows_to_write:
        writer.writerows(rows_to_write)

print(f'Total pseudo labels generated: {count}')

loading word2vec model ...
total words: 112614
Total pseudo labels generated: 108258


# Train and test with pseudo labeled data

In [47]:
# Split Dataset
train_idx, train_label_text, label = load_train_label('dataset/train_label.csv')
pseudo_idx, pseudo_label_text, pseudo_label = load_train_label('pseudo_labels.csv')

train_nolabel_text = load_train_nolabel('dataset/train_nolabel.csv')

w2v_sentences = train_label_text + train_nolabel_text

# Use labeled data for Word2Vec embeddings (# TODO: Perform unsupervised Learning for w2v)
preprocessor = Preprocessor(w2v_sentences, w2v_config)

train_idx, valid_idx, train_label_text, valid_label_text, train_label, valid_label = train_test_split(train_idx, train_label_text, label, test_size=0.12)
train_idx += pseudo_idx
train_label_text += pseudo_label_text
train_label += pseudo_label
train_dataset, valid_dataset = TwitterDataset(train_idx, train_label_text, train_label, preprocessor, train_dataset_dropout), TwitterDataset(valid_idx, valid_label_text, valid_label, preprocessor)

test_idx, test_text = load_test('dataset/test.csv')
test_dataset = TwitterDataset(test_idx, test_text, None, preprocessor)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True,
                                            collate_fn = train_dataset.collate_fn,
                                            num_workers = 8)
valid_loader = torch.utils.data.DataLoader(dataset = valid_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = False,
                                            collate_fn = valid_dataset.collate_fn,
                                            num_workers = 8)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = False,
                                            collate_fn = test_dataset.collate_fn,
                                            num_workers = 8)

# Instantiate The Model
backbone = LSTM_Backbone(preprocessor.embedding_matrix, **lstm_config)
header = Header(**header_config)

# Run Training
MODEL_DIR2 = 'model_with_pseudo.pth'
run_training(train_loader, valid_loader, backbone, header, EPOCH_NUM, lr, device, MODEL_DIR2)

# Run Testing
run_testing(test_loader, backbone, header, device, 'pred.csv')

loading word2vec model ...
total words: 112614
[Epoch 1] Train loss:0.584 acc:65.989 | Val loss:0.420 acc:80.687 | LR:0.001000
New best model saved with accuracy: 80.687
[Epoch 2] Train loss:0.557 acc:67.907 | Val loss:0.416 acc:81.694 | LR:0.001000
New best model saved with accuracy: 81.694
[Epoch 3] Train loss:0.549 acc:68.370 | Val loss:0.397 acc:82.331 | LR:0.001000
New best model saved with accuracy: 82.331
[Epoch 4] Train loss:0.541 acc:68.892 | Val loss:0.391 acc:82.332 | LR:0.001000
New best model saved with accuracy: 82.332
[Epoch 5] Train loss:0.533 acc:69.290 | Val loss:0.387 acc:82.585 | LR:0.001000
New best model saved with accuracy: 82.585
[Epoch 6] Train loss:0.524 acc:69.781 | Val loss:0.397 acc:82.541 | LR:0.001000
[Epoch 7] Train loss:0.515 acc:70.258 | Val loss:0.407 acc:82.109 | LR:0.001000
[Epoch 8] Train loss:0.504 acc:70.830 | Val loss:0.426 acc:82.148 | LR:0.001000
[Epoch 9] Train loss:0.482 acc:71.928 | Val loss:0.435 acc:81.857 | LR:0.000500
[Epoch 10] Train l